import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [1]:
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when

findspark.init()

create local SparkSession

In [2]:
%%time
spark = (
    SparkSession.builder
        .appName("Task 1")
        .getOrCreate()
)

25/01/27 10:35:32 WARN Utils: Your hostname, rewinteer-ThinkPad-T480 resolves to a loopback address: 127.0.1.1; using 10.202.33.209 instead (on interface wlp3s0)
25/01/27 10:35:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/27 10:35:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


CPU times: user 19.1 ms, sys: 13.5 ms, total: 32.7 ms
Wall time: 3.21 s


read csv with inferschema

In [3]:
%%time
df = spark.read.options(inferSchema=True, header=True).csv('ds_salaries.csv')

CPU times: user 2.36 ms, sys: 2.23 ms, total: 4.59 ms
Wall time: 4.76 s


read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [4]:
%%time
df = spark.read.options(inferSchema=True, header=True).csv('ds_salaries.csv')

CPU times: user 2.98 ms, sys: 919 μs, total: 3.9 ms
Wall time: 393 ms


write schema of scv on screen

In [5]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



create schema of this scv

In [6]:
schema = StructType() \
    .add('id', IntegerType()) \
    .add('work_year', IntegerType()) \
    .add('experience_level', StringType()) \
    .add('employment_type', StringType()) \
    .add('job_title', StringType()) \
    .add('salary', IntegerType()) \
    .add('salary_currency', StringType()) \
    .add('salary_in_usd', IntegerType()) \
    .add('employee_residence', StringType()) \
    .add('remote_ratio', IntegerType()) \
    .add('company_location', StringType()) \
    .add('company_size', StringType())

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [7]:
%%time
df = spark.read.format('csv') \
    .option('header', True) \
    .schema(schema) \
    .load('ds_salaries.csv')

CPU times: user 4.84 ms, sys: 10 μs, total: 4.85 ms
Wall time: 60.1 ms


this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [8]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [9]:
df.show

<bound method DataFrame.show of DataFrame[id: int, work_year: int, experience_level: string, employment_type: string, job_title: string, salary: int, salary_currency: string, salary_in_usd: int, employee_residence: string, remote_ratio: int, company_location: string, company_size: string]>

print data in dataframe using display(df.toPandas())

In [10]:
display(df.toPandas())

25/01/27 10:36:19 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
 Schema: id, work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
Expected: id but found: 
CSV file: file:///home/rewinteer/Documents/Projects/PycharmProjects/Innowise/task-4/PySpark_Basics/ds_salaries.csv


,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [11]:
df_job_title = df.select('job_title').distinct()

print all rows from df_job_titles without truncating jobs

In [12]:
# display(df_job_title.toPandas())
df_job_title.show(df_job_title.count(), truncate=0)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
|Research S

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [13]:
df_analytic = df.groupBy('job_title').agg(
    max('salary_in_usd').alias('max_salary'), \
    avg('salary_in_usd').alias('avg_salary'), \
    min('salary_in_usd').alias('min_salary')
).orderBy('job_title')

print all rows from df_analytic without trancating jobs

In [14]:
df_analytic.show(df_analytic.count(), truncate=False)

+----------------------------------------+----------+------------------+----------+
|job_title                               |max_salary|avg_salary        |min_salary|
+----------------------------------------+----------+------------------+----------+
|3D Computer Vision Researcher           |5409      |5409.0            |5409      |
|AI Scientist                            |200000    |66135.57142857143 |12000     |
|Analytics Engineer                      |205300    |175000.0          |135000    |
|Applied Data Scientist                  |380000    |175655.0          |54238     |
|Applied Machine Learning Scientist      |423000    |142068.75         |31875     |
|BI Data Analyst                         |150000    |74755.16666666667 |9272      |
|Big Data Architect                      |99703     |99703.0           |99703     |
|Big Data Engineer                       |114047    |51974.0           |5882      |
|Business Data Analyst                   |135000    |76691.2           |1844

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [15]:
w = Window.orderBy(df_analytic.avg_salary.desc())
df_analytic = df_analytic.withColumn("row_id", row_number().over(w))


print all data from df_analytic

In [16]:
df_analytic.show(df_analytic.count(), truncate=False)

25/01/27 10:37:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/27 10:37:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/27 10:37:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/27 10:37:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/27 10:37:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/27 10:37:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------------------------------------+----------+------------------+----------+------+
|job_title                               |max_salary|avg_salary        |min_salary|row_id|
+----------------------------------------+----------+------------------+----------+------+
|Data Analytics Lead                     |405000    |405000.0          |405000    |1     |
|Principal Data Engineer                 |600000    |328333.3333333333 |185000    |2     |
|Financial Data Analyst                  |450000    |275000.0          |100000    |3     |
|Principal Data Scientist                |416000    |215242.42857142858|148261    |4     |
|Director of Data Science                |325000    |195074.0          |130026    |5     |
|Data Architect                          |266400    |177873.9090909091 |90700     |6     |
|Applied Data Scientist                  |380000    |175655.0          |54238     |7     |
|Analytics Engineer                      |205300    |175000.0          |135000    |8     |

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [17]:
new_arrangement = ('row_id', 'job_title', 'max_salary', 'avg_salary', 'min_salary')
df_analytic = df_analytic.select(*new_arrangement)

print df_analytic now

In [18]:
df_analytic.show(df_analytic.count(), truncate=False)

25/01/27 10:37:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/27 10:37:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/27 10:37:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/27 10:37:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/27 10:37:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/27 10:37:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------+----------------------------------------+----------+------------------+----------+
|row_id|job_title                               |max_salary|avg_salary        |min_salary|
+------+----------------------------------------+----------+------------------+----------+
|1     |Data Analytics Lead                     |405000    |405000.0          |405000    |
|2     |Principal Data Engineer                 |600000    |328333.3333333333 |185000    |
|3     |Financial Data Analyst                  |450000    |275000.0          |100000    |
|4     |Principal Data Scientist                |416000    |215242.42857142858|148261    |
|5     |Director of Data Science                |325000    |195074.0          |130026    |
|6     |Data Architect                          |266400    |177873.9090909091 |90700     |
|7     |Applied Data Scientist                  |380000    |175655.0          |54238     |
|8     |Analytics Engineer                      |205300    |175000.0          |135000    |

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [19]:
df_exp_lvl = df.groupBy('experience_level').agg(max('salary_in_usd').alias('salary_in_usd'))

print here df_exp_lvl

In [20]:
df_exp_lvl.show(df_exp_lvl.count(), truncate=False)

+----------------+-------------+
|experience_level|salary_in_usd|
+----------------+-------------+
|EX              |600000       |
|MI              |450000       |
|EN              |250000       |
|SE              |412000       |
+----------------+-------------+



create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [22]:
columns = ('id', 'experience_level', 'salary_in_usd', 'employee_residence')
join_columns = ['experience_level', 'salary_in_usd']
df_best = df.join(df_exp_lvl, on=join_columns).select(*columns)

print df_best

In [23]:
df_best.show(df_best.count(), truncate=False)

+---+----------------+-------------+------------------+
|id |experience_level|salary_in_usd|employee_residence|
+---+----------------+-------------+------------------+
|33 |MI              |450000       |US                |
|37 |EN              |250000       |US                |
|63 |SE              |412000       |US                |
|97 |MI              |450000       |US                |
|252|EX              |600000       |US                |
+---+----------------+-------------+------------------+



25/01/27 10:38:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , experience_level, salary_in_usd, employee_residence
 Schema: id, experience_level, salary_in_usd, employee_residence
Expected: id but found: 
CSV file: file:///home/rewinteer/Documents/Projects/PycharmProjects/Innowise/task-4/PySpark_Basics/ds_salaries.csv


drop duplicates if exist by experience_level

In [24]:
df_best = df_best.dropDuplicates(['experience_level'])

print df_best

In [25]:
df_best.show(df_best.count(), truncate=False)

+---+----------------+-------------+------------------+
|id |experience_level|salary_in_usd|employee_residence|
+---+----------------+-------------+------------------+
|37 |EN              |250000       |US                |
|252|EX              |600000       |US                |
|33 |MI              |450000       |US                |
|63 |SE              |412000       |US                |
+---+----------------+-------------+------------------+



25/01/27 10:38:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , experience_level, salary_in_usd, employee_residence
 Schema: id, experience_level, salary_in_usd, employee_residence
Expected: id but found: 
CSV file: file:///home/rewinteer/Documents/Projects/PycharmProjects/Innowise/task-4/PySpark_Basics/ds_salaries.csv


create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [26]:
df_new_best = df_best.withColumn("experience_level", when(df.experience_level == "MI","Middle")
                                 .when(df.experience_level == "SE","Senior")
                                 .otherwise('Null'))

print df_new_best

In [27]:
df_new_best.show(df_new_best.count(), truncate=False)

+---+----------------+-------------+------------------+
|id |experience_level|salary_in_usd|employee_residence|
+---+----------------+-------------+------------------+
|37 |Null            |250000       |US                |
|252|Null            |600000       |US                |
|33 |Middle          |450000       |US                |
|63 |Senior          |412000       |US                |
+---+----------------+-------------+------------------+



25/01/27 10:38:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , experience_level, salary_in_usd, employee_residence
 Schema: id, experience_level, salary_in_usd, employee_residence
Expected: id but found: 
CSV file: file:///home/rewinteer/Documents/Projects/PycharmProjects/Innowise/task-4/PySpark_Basics/ds_salaries.csv


write df_new_best like 1.csv and load then it to df_final

In [33]:
df_new_best.toPandas().to_csv("1.csv")
df_final = spark.read.options(header=True).csv('1.csv')

print df_final

In [34]:
df_final.show(df_final.count(), truncate=False)

+---+---+----------------+-------------+------------------+
|_c0|id |experience_level|salary_in_usd|employee_residence|
+---+---+----------------+-------------+------------------+
|0  |37 |Null            |250000       |US                |
|1  |252|Null            |600000       |US                |
|2  |33 |Middle          |450000       |US                |
|3  |63 |Senior          |412000       |US                |
+---+---+----------------+-------------+------------------+



25/01/27 10:44:37 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, experience_level, salary_in_usd, employee_residence
 Schema: _c0, id, experience_level, salary_in_usd, employee_residence
Expected: _c0 but found: 
CSV file: file:///home/rewinteer/Documents/Projects/PycharmProjects/Innowise/task-4/PySpark_Basics/1.csv


filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [39]:
df_final = df_final.filter(df_final['experience_level'] != 'Null').join(df, on=['salary_in_usd', 'employee_residence'])

print df_final

In [40]:
df_final.show(df_final.count(), truncate=False)

+-------------+------------------+---+---+----------------+---+---------+----------------+---------------+----------------------+------+---------------+------------+----------------+------------+
|salary_in_usd|employee_residence|_c0|id |experience_level|id |work_year|experience_level|employment_type|job_title             |salary|salary_currency|remote_ratio|company_location|company_size|
+-------------+------------------+---+---+----------------+---+---------+----------------+---------------+----------------------+------+---------------+------------+----------------+------------+
|450000       |US                |2  |33 |Middle          |33 |2020     |MI              |FT             |Research Scientist    |450000|USD            |0           |US              |M           |
|412000       |US                |3  |63 |Senior          |63 |2020     |SE              |FT             |Data Scientist        |412000|USD            |100         |US              |L           |
|450000       |US   

25/01/27 10:58:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, experience_level, salary_in_usd, employee_residence
 Schema: _c0, id, experience_level, salary_in_usd, employee_residence
Expected: _c0 but found: 
CSV file: file:///home/rewinteer/Documents/Projects/PycharmProjects/Innowise/task-4/PySpark_Basics/1.csv
25/01/27 10:58:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
 Schema: id, work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
Expected: id but found: 
CSV file: file:///home/rewinteer/Documents/Projects/PycharmProjects/Innowise/task-4/PySpark_Basics/ds_salaries.csv


last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [56]:
var = df_final.select(max('salary_in_usd')).collect()[0][0]
var

'450000'

It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system